In [1]:

import sys, os 
sys.path.insert(0, '/Users/anis/test_labnbook/Indicators/scripts')  
import json, gzip,ujson
import pandas as pd 
from  utils.utils  import get_authors_changes
import numpy as np, toml
from typing import Tuple, List, Dict
import mysql.connector as mariadb
import warnings    
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows',700)
import matplotlib.pyplot as plt 

from flow_3 import * 
from utils.utils import *
import seaborn as sns 

# password = "11950022"
# host = "localhost"
# database_name  = "Labnbook"
# user = 'root'
# try:
#     conn = mariadb.connect(user=user, password=password,
#                            host=host, database=database_name)
# except mariadb.Error as e:
#     print(f"Error connecting to MariaDB Platform: {e}")

# # Récupération du curseur
# cursor = conn.cursor()
# trace = pd.read_sql(
#     " SELECT id_trace, id_labdoc,id_user ,id_action, action_time from trace Order By id_labdoc ASC, action_time ASC", conn)
# # WHERE  id_action=9 

In [2]:
# selected_labdoc =445494
# # Import all summary
# summary_all  = pd.read_csv('/Users/anis/test_labnbook/Indicators/data/tmp/reports/3_summary_all.csv', index_col = [0])
# summary_labdoc = summary_all[summary_all["id_labdoc"] == int(selected_labdoc)]
# summary_labdoc.drop(['id_mission','id_report','id_labdoc'],axis =1, inplace = True)
# summary_labdoc['action_time'] = pd.to_datetime(summary_labdoc['action_time'])
# summary_labdoc['edition_time'] = summary_labdoc['action_time'].diff()
# summary_labdoc["id_trace"] = summary_labdoc["id_trace"].astype(str)
# summary_labdoc.set_index('id_trace',inplace=True)
# summary_labdoc['1-sim'] = 1 - summary_labdoc["sim"]
# edition_time = summary_labdoc['edition_time']
# summary_labdoc["edition_time"] = summary_labdoc["edition_time"].astype(str)
# summary_labdoc["action_time"] = summary_labdoc["action_time"].astype(str)

# times_all = pd.read_csv("/Users/anis/test_labnbook/Indicators/data/tmp/reports/3_times.csv",index_col=[0])
# times_labdoc = times_all[times_all["id_labdoc_x"]==selected_labdoc]
# times_labdoc.rename(columns={"id_labdoc_x": "id_labdoc"},inplace =True)
# times_labdoc.drop(["id_labdoc_y"], axis = 1 , inplace = True)
# times_labdoc = times_labdoc.set_index("id_trace")
# times_labdoc.index = times_labdoc.index.astype(str)
# times_labdoc_thin = times_labdoc[["n_modify_id","effective_time","teacher"]]
# pd.merge(times_labdoc_thin,summary_labdoc, left_index=True, right_index=True)

# Data transformation

In [3]:
summary_all  = pd.read_csv('/Users/anis/test_labnbook/Indicators/data/tmp/reports/3_summary_all.csv', index_col = [0])
times_all = pd.read_csv("/Users/anis/test_labnbook/Indicators/data/tmp/reports/3_times.csv",index_col=[0])
times_all.drop(['id_labdoc_x'], axis = 1, inplace=True)
times_all.rename(columns={"id_labdoc_y" : "id_labdoc"},inplace=True)
summary_all["sim_evolu"] = 1 - summary_all["sim"]



# Fusionner les deux dataframes
df = pd.merge(summary_all,times_all,on="id_trace",how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

# pd.merge(df_1,df_2, on='id_trace',how="outer", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
nouvel_ordre_colonnes = ['id_mission', 'id_report', 'id_labdoc','id_trace','id_user','n_users','teacher','n_segments','n_tokens','action_time',"effective_time",'n_modify_id','eqc','coec','sim_evolu']  # Définir le nouvel ordre des colonnes
df = df[nouvel_ordre_colonnes]

# Convertir le dtype de certaines colonnes
integer_col = ["id_mission","id_report","id_labdoc","id_user","n_segments","n_tokens","n_modify_id"]
df[integer_col] = df[integer_col].astype('Int64')
df.teacher = df.teacher.astype('bool')
df.head(5)

,id_mission,id_report,id_labdoc,id_trace,id_user,n_users,teacher,n_segments,n_tokens,action_time,effective_time,n_modify_id,eqc,coec,sim_evolu
0,451,35694,323357,5455308,10706,1,False,1,32,2020-11-16 09:11:02,540.0,18,0.0,0.00,0.00
1,451,35694,323357,5457355,10727,2,False,2,37,2020-11-16 10:07:17,330.0,11,0.8,0.15,0.48
2,451,35694,323370,5461072,10727,1,False,8,203,2020-11-16 11:36:42,1050.0,35,0.0,0.00,0.00
3,451,35694,323362,5462296,10727,1,False,3,68,2020-11-16 12:13:46,1170.0,39,0.0,0.00,0.00
4,451,35694,323366,5459926,10727,1,False,5,134,2020-11-16 11:09:48,540.0,18,0.0,0.00,0.00


* Il y'a plus de lignes dans times_all que summary_all car dans le premier on a pas les traces des enseignants.

In [4]:
df[df['id_labdoc']==445494]


,id_mission,id_report,id_labdoc,id_trace,id_user,n_users,teacher,n_segments,n_tokens,action_time,effective_time,n_modify_id,eqc,coec,sim_evolu
7028,647,45220,445494,0,<NA>,1,True,1,86,NaN,NaN,<NA>,0.000000,0.000000,0.00
7180,647,45220,445494,7481881,12673,2,True,3,112,2021-03-16 10:05:29,690.0,23,0.000000,0.000000,0.31
7181,647,45220,445494,7482763,12661,3,True,3,111,2021-03-16 11:18:11,120.0,4,0.042793,0.045333,0.00
7182,647,45220,445494,7482817,12673,3,True,6,121,2021-03-16 11:21:13,420.0,14,0.066413,0.070133,0.00
7183,647,45220,445494,7482845,12661,3,True,6,121,2021-03-16 11:22:42,60.0,2,0.066413,0.070133,0.00
7184,647,45220,445494,7505160,12673,3,True,8,150,2021-03-18 08:49:40,1740.0,58,0.118471,0.150075,0.02


# Data analysis 

In [5]:
# # Prendre la derniére valeur de chaque labdoc 
# df_last = df.groupby('id_labdoc').last().reset_index()
# df_last.head()
def tests(scores = list)->int : 
   n_pass =  np.arange(0, len(scores) )
   n_pass[0] = 1
   res =  np.cumsum(scores) /n_pass
   return np.round(res,2) 

In [6]:
ex_1 = [0,1,0.5,0]

ex_2 = [0,1,1,1]
ex_3 = [0,0,0,0]
ex_4 = [0,1,0,1]

print("\nexample 1:",tests(ex_1),"\nexample 2:",tests(ex_2),"\nexample 3:",tests(ex_3),"\nexample 4:",tests(ex_4))


example 1: [0.   1.   0.75 0.5 ] 
example 2: [0. 1. 1. 1.] 
example 3: [0. 0. 0. 0.] 
example 4: [0.   1.   0.5  0.67]


In [7]:
# Access data of a specific group
res = {}
for id_labdoc in df["id_labdoc"].unique():
    res[id_labdoc] = (tests(df[df['id_labdoc']==id_labdoc]["sim_evolu"].values))

# Conversion du dictionnaire en liste de tuples
data_list = [(key, value) for key, value in res.items()]
# Création du dataframe
df_bis = pd.DataFrame(data_list, columns=['id_labdoc', 'sim_idx'])

# Explosion des valeurs de l'array
df_bis = df_bis.explode('sim_idx')
# Réinitialisation de l'index
df_bis["sim_idx"] = df_bis["sim_idx"].astype('float')

df_bis = df_bis.reset_index(drop=True)
df = df.reset_index(drop=True)
# Affichage du dataframe
# df["sim_idx"] = df_bis["sim_idx"]

In [24]:
id_labdoc = 445494
df[df['id_labdoc']==id_labdoc]

,id_mission,id_report,id_labdoc,id_trace,id_user,n_users,teacher,n_segments,n_tokens,action_time,effective_time,n_modify_id,eqc,coec,sim_evolu
7028,647,45220,445494,0,<NA>,1,True,1,86,NaN,NaN,<NA>,0.000000,0.000000,0.00
7180,647,45220,445494,7481881,12673,2,True,3,112,2021-03-16 10:05:29,690.0,23,0.000000,0.000000,0.31
7181,647,45220,445494,7482763,12661,3,True,3,111,2021-03-16 11:18:11,120.0,4,0.042793,0.045333,0.00
7182,647,45220,445494,7482817,12673,3,True,6,121,2021-03-16 11:21:13,420.0,14,0.066413,0.070133,0.00
7183,647,45220,445494,7482845,12661,3,True,6,121,2021-03-16 11:22:42,60.0,2,0.066413,0.070133,0.00
7184,647,45220,445494,7505160,12673,3,True,8,150,2021-03-18 08:49:40,1740.0,58,0.118471,0.150075,0.02


In [10]:
df_bis[df_bis['id_labdoc']==id_labdoc]

,id_labdoc,sim_idx
7064,445494,0.00
7065,445494,0.31
7066,445494,0.16
7067,445494,0.10
7068,445494,0.08
7069,445494,0.07


In [18]:
(df_bis['id_labdoc'] == df["id_labdoc"]).mean()

0.8011494252873563

In [25]:
df[df['id_labdoc']==id_labdoc]

,id_mission,id_report,id_labdoc,id_trace,id_user,n_users,teacher,n_segments,n_tokens,action_time,effective_time,n_modify_id,eqc,coec,sim_evolu
7028,647,45220,445494,0,<NA>,1,True,1,86,NaN,NaN,<NA>,0.000000,0.000000,0.00
7180,647,45220,445494,7481881,12673,2,True,3,112,2021-03-16 10:05:29,690.0,23,0.000000,0.000000,0.31
7181,647,45220,445494,7482763,12661,3,True,3,111,2021-03-16 11:18:11,120.0,4,0.042793,0.045333,0.00
7182,647,45220,445494,7482817,12673,3,True,6,121,2021-03-16 11:21:13,420.0,14,0.066413,0.070133,0.00
7183,647,45220,445494,7482845,12661,3,True,6,121,2021-03-16 11:22:42,60.0,2,0.066413,0.070133,0.00
7184,647,45220,445494,7505160,12673,3,True,8,150,2021-03-18 08:49:40,1740.0,58,0.118471,0.150075,0.02
